In [ ]:
!pip3 install -U transformers
!pip3 install datasets
!pip3 install sentencepiece
!pip3 install seqeval


In [ ]:
!pip3 install sacremoses

In [ ]:
!pip3 install accelerate
import accelerate

In [ ]:
from datasets import ClassLabel, load_dataset, load_metric, DownloadMode

In [ ]:
raw_datasets = load_dataset('ai4bharat/naamapadam', 'or')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
raw_datasets
subset_size = 10000
subset_indices = range(subset_size)

# Update the training dataset with the subset
raw_datasets['train'] = raw_datasets['train'].select(subset_indices)

In [ ]:
column_names = raw_datasets["train"].column_names

In [ ]:
features = raw_datasets["train"].features
text_column_name = "tokens"
label_column_name = "ner_tags"

In [ ]:
label_list = features[label_column_name].feature.names

label_to_id = {label_list[i]: features[label_column_name].feature.str2int( label_list[i] ) for i in range(len(label_list))}


num_labels = len(label_list)

In [ ]:
from transformers import AutoModelForTokenClassification, AutoConfig, AutoTokenizer, TrainingArguments, Trainer, DataCollatorForTokenClassification, EarlyStoppingCallback, IntervalStrategy
import numpy as np

config = AutoConfig.from_pretrained('xlnet-base-cased', num_labels=num_labels, finetuning_task='ner')
tokenizer = AutoTokenizer.from_pretrained("xlnet-base-cased")
model = AutoModelForTokenClassification.from_pretrained('xlnet-base-cased', num_labels=num_labels )

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Some weights of XLNetForTokenClassification were not initialized from the model checkpoint at xlnet-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
ymodel=model.to('cuda')

In [ ]:
# Tokenize all texts and align the labels with them.
padding = "max_length"
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(
        examples[text_column_name],
        padding=padding,
        truncation=True,
        max_length=512,
        # We use this argument because the texts in our dataset are lists of words (with a label for each word).
        is_split_into_words=True,
    )
    labels = []
    for i, label in enumerate(examples[label_column_name]):
        # print('=====')
        # print('{} {}'.format(i,label)) #ak
        word_ids = tokenized_inputs.word_ids(batch_index=i)

        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            # Special tokens have a word id that is None. We set the label to -100 so they are automatically
            # ignored in the loss function.
            if word_idx is None:
                label_ids.append(-100)
            # We set the label for the first token of each word.
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
            # For the other tokens in a word, we set the label to either the current label or -100, depending on
            # the label_all_tokens flag.
            else:
                label_ids.append(-100)
            previous_word_idx = word_idx

        labels.append(label_ids)
    tokenized_inputs["labels"] = labels
    return tokenized_inputs

In [ ]:
train_dataset = raw_datasets["train"]
train_dataset = train_dataset.map(
    tokenize_and_align_labels,
    batched=True,
    num_proc=4,
    load_from_cache_file=True,
    desc="Running tokenizer on train dataset",
)

Running tokenizer on train dataset (num_proc=4):   0%|          | 0/10000 [00:00<?, ? examples/s]

In [ ]:
eval_dataset = raw_datasets["validation"]
eval_dataset = eval_dataset.map(
    tokenize_and_align_labels,
    batched=True,
    num_proc=4,
    load_from_cache_file=True,
    desc="Running tokenizer on Validation dataset",
)

Running tokenizer on Validation dataset (num_proc=4):   0%|          | 0/993 [00:00<?, ? examples/s]

In [ ]:
data_collator = DataCollatorForTokenClassification(tokenizer)

In [ ]:
metric = load_metric("seqeval")

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    # Remove ignored index (special tokens)
    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = metric.compute(predictions=true_predictions, references=true_labels)
    # Unpack nested dictionaries
    final_results = {}
    for key, value in results.items():
        if isinstance(value, dict):
            for n, v in value.items():
                final_results[f"{key}_{n}"] = v
        else:
            final_results[key] = value
    return final_results

<ipython-input-16-c48fc96d3f0c>:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("seqeval")
/usr/local/lib/python3.10/dist-packages/datasets/load.py:752: FutureWarning: The repository for seqeval contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.16.1/metrics/seqeval/seqeval.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


In [ ]:
args=TrainingArguments(
    output_dir='output_dir',
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,)

In [ ]:
trainer = Trainer(
    model=model,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    # callbacks=[early_stopping_callback],
    args=args,
)

In [ ]:
train_result = trainer.train()
metrics = train_result.metrics

You're using a XLNetTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
500,0.585200
1000,0.603000
1500,0.577200
2000,0.549000
2500,0.526000
3000,0.532800
3500,0.509800


In [ ]:

metrics = trainer.evaluate()

trainer.log_metrics("eval", metrics)

/usr/local/lib/python3.10/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


***** eval metrics *****
  epoch                   =        3.0
  eval_LOC_f1             =      0.029
  eval_LOC_number         =        268
  eval_LOC_precision      =        0.5
  eval_LOC_recall         =     0.0149
  eval_ORG_f1             =        0.0
  eval_ORG_number         =        229
  eval_ORG_precision      =        0.0
  eval_ORG_recall         =        0.0
  eval_PER_f1             =     0.0636
  eval_PER_number         =        386
  eval_PER_precision      =     0.1744
  eval_PER_recall         =     0.0389
  eval_loss               =     0.4629
  eval_overall_accuracy   =      0.883
  eval_overall_f1         =     0.0389
  eval_overall_precision  =     0.2021
  eval_overall_recall     =     0.0215
  eval_runtime            = 0:01:53.32
  eval_samples_per_second =      8.763
  eval_steps_per_second   =      1.103


In [ ]:
metrics

{'eval_loss': 0.46294641494750977,
 'eval_LOC_precision': 0.5,
 'eval_LOC_recall': 0.014925373134328358,
 'eval_LOC_f1': 0.02898550724637681,
 'eval_LOC_number': 268,
 'eval_ORG_precision': 0.0,
 'eval_ORG_recall': 0.0,
 'eval_ORG_f1': 0.0,
 'eval_ORG_number': 229,
 'eval_PER_precision': 0.1744186046511628,
 'eval_PER_recall': 0.038860103626943004,
 'eval_PER_f1': 0.0635593220338983,
 'eval_PER_number': 386,
 'eval_overall_precision': 0.20212765957446807,
 'eval_overall_recall': 0.021517553793884484,
 'eval_overall_f1': 0.038894575230296824,
 'eval_overall_accuracy': 0.8830373545621556,
 'eval_runtime': 113.3227,
 'eval_samples_per_second': 8.763,
 'eval_steps_per_second': 1.103,
 'epoch': 3.0}

In [ ]:
import json

In [ ]:
file_path = "evaluation_metrics.txt"

# Save metrics to a txt file
with open(file_path, "a") as txt_file:  # Use "a" for append mode
        txt_file.write(f"Language Name - Assamese")

print(f"Metrics saved to {file_path}")

Metrics saved to evaluation_metrics.txt
